# CTOOLS EXAMPLE

#### In order to start this notebook you need to have sourced both gammalib and the ctools: you can skip the first box if you have already set up both gammalib and ctools by launching the gammalib_ctools.sh script before using this notebook.

#### The script simply export the right environmental variables and works only if the file gammalib_ctools.sh is in /home/fermi-cta.

In [1]:
# export GAMMALIB=/usr/local/gamma
# source $GAMMALIB/bin/gammalib-init.sh
# export CTOOLS=/usr/local/gamma
# source $CTOOLS/bin/ctools-init.sh

!source /home/fermi-cta/gammalib_ctools.sh
!more /home/fermi-cta/gammalib_ctools.sh 

export GAMMALIB=/usr/local/gamma
source $GAMMALIB/bin/gammalib-init.sh
#
export CTOOLS=/usr/local/gamma
source $CTOOLS/bin/ctools-init.sh
#
export GAMMAPY_EXTRA=~/gammapy-extra
 


In [2]:
import gammalib
import ctools
from cscripts import csobsdef
import csv

## Create Pointing definition ASCII file in comma-separated value (CSV) format

### We set up everything we need in order to create the .csv file that will we used later on in order to create the xml file with the script csobsdef. The xml file which is created with this script can be given to the simulation tool (ctobssim). 

In [25]:
# the id must be different for each observation

csv_filename = 'obs_test.csv'
name = 'Crab'
ra = 83.6331
dec = 22.0145
duration = 30
emin = 0.05
emax = 20.0
rad = 8.0
caldb = 'prod2'
irf = 'South_0.5h'
deadc = 0.95

with open(csv_filename, 'w') as csvfile:
    #fieldnames = ['name', 'id','ra','dec','duration','emin', 'emax', 'rad', 'deadc', 'caldb', 'irf']
    fieldnames = ['name', 'id','ra','dec','duration', 'emin', 'emax','']

    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for id in range(1,3):
        if id < 10:
            id_print = "000"+str(id)
        elif id < 100:
            id_print = "00"+str(id)
        #writer.writerow({'name': name, 'id': id_print,'ra' : ra, 'dec':dec,'duration':duration, 'emin':emin, 'emax':emax, 'rad':rad, 'deadc':deadc, 'caldb':caldb, 'irf':irf})
        writer.writerow({'name': name, 'id': id_print,'ra' : ra, 'dec':dec,'duration':duration, 'emin':emin, 'emax':emax})

In [26]:
# this is the csv file: see http://cta.irap.omp.eu/ctools/users/reference_manual/csobsdef.html for details.
!more obs_test.csv

name,id,ra,dec,duration,emin,emax,
Crab,0001,83.6331,22.0145,30,0.05,20.0,
Crab,0002,83.6331,22.0145,30,0.05,20.0,


## Create Observation Definition XML file

All the tools can be used from Python in the same way. First you have to create an **instance** of the the class, then you have to set up all the required parameters. If the python script is run from terminal (*$ python script.py*) and some informations are missing, they will be asked by the script itself (CHECK).

The execute() method executes the ctobssim class in the same way as if it were executed from the command line. Alternatively, you may “run” the ctobssim tool using run(). The main difference to the execute() method is that the **run() will not write** the simulated event file to disk. Why is this useful? Well, after having typed sim.run() the ctobssim class still exists as an object in memory, including all the simulated events.

In [28]:
# the script is called in this way because it was explicitely imported at the beginning

obsdef = csobsdef()                #create empty instance
obsdef['inpnt'] = csv_filename     #Pointing definition ASCII file in comma-separated value (CSV) format.
#obsdef['emin'] = emin 
#obsdef['emax'] = emax
obsdef['deadc'] = deadc
obsdef['caldb'] = caldb
obsdef['irf'] = irf
obsdef['rad'] = rad
obsdef['outobs'] = 'out_csobsdef.xml'
obsdef.execute()

In [29]:
#visualization of the Observation Definition XML file.
!xmllint 'out_csobsdef.xml'

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<observation_list title="observation list">
  <observation name="Crab" id="0001" instrument="CTA">
    <parameter name="Pointing" ra="83.6331" dec="22.0145"/>
    <parameter name="EnergyBoundaries" emin="50000" emax="20000000"/>
    <parameter name="GoodTimeIntervals" tmin="0" tmax="30"/>
    <parameter name="TimeReference" mjdrefi="51544" mjdreff="0.5" timeunit="s" timesys="TT" timeref="LOCAL"/>
    <parameter name="RegionOfInterest" ra="83.6331" dec="22.0145" rad="8"/>
    <parameter name="Deadtime" deadc="0.95"/>
    <parameter name="Calibration" database="prod2" response="South_0.5h"/>
  </observation>
  <observation name="Crab" id="0002" instrument="CTA">
    <parameter name="Pointing" ra="83.6331" dec="22.0145"/>
    <parameter name="EnergyBoundaries" emin="50000" emax="20000000"/>
    <parameter name="GoodTimeIntervals" tmin="30" tmax="60"/>
    <parameter name="TimeReference" mjdrefi="51544" mjdreff="0.5" timeunit="s" times

## Simulation (ctobssim)

Now that we have created the **observation definition .xml** file, we can run the simulation. We only need to provide a **model** for the source that we want to simulate. We have to give a model for the source to be simulated: since many sources are simulated together, the xml model will have a piece for every source to simulate, including the background.

**RUN CTOBBSIM**

Everything is kept in memory in the object sim.

In [18]:
sim = ctools.ctobssim()
sim["inobs"] = "out_csobsdef.xml"
sim["inmodel"] = "${CTOOLS}/share/models/crab.xml"   #we load a sample model
sim.run()

# ...and print the results
print(sim.obs())
print(sim.obs()[0].events())   #for the first pointing (id = 0001)
print(sim.obs()[1].events())   #for the second pointing (id = 0002)...and so on

**EXECUTE CTOBBSIM**

The object sim in created and destroyed once the method execute() is called. This will save a fits file for every pointing and a global *observation definition XML file*, different from the output of csobsdef (out_csobsdef.xml), which has a observation item for each pointing, in which the name of the fits file of the events is specified. 

In [11]:
# EXECUTE CTOBBSIM
sim = ctools.ctobssim()
sim["inobs"] = "out_csobsdef.xml"
sim["inmodel"] = "${CTOOLS}/share/models/crab.xml"  #we load a sample model
sim["prefix"] = name+"_obs_out_"                    # this is optional: "sim_events_" is the default
sim["outevents"] = name+"_obs_out.xml"
sim.execute()

In [12]:
!xmllint *_obs_out.xml
#like = ctools.ctlike(sim_cp.obs())


<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<observation_list title="observation list">
  <observation name="Crab" id="0001" instrument="CTA">
    <parameter name="EventList" file="Crab_obs_out_000001.fits"/>
    <parameter name="Calibration" database="prod2" response="South_0.5h"/>
  </observation>
  <observation name="Crab" id="0002" instrument="CTA">
    <parameter name="EventList" file="Crab_obs_out_000002.fits"/>
    <parameter name="Calibration" database="prod2" response="South_0.5h"/>
  </observation>
</observation_list>


In [9]:
!xmllint $CTOOLS/share/models/crab.xml

<?xml version="1.0" standalone="no"?>
<source_library title="source library">
  <source name="Crab" type="PointSource">
    <spectrum type="PowerLaw">
       <parameter name="Prefactor" scale="1e-16" value="5.7" min="1e-07" max="1000.0" free="1"/>
       <parameter name="Index" scale="-1" value="2.48" min="0.0" max="+5.0" free="1"/>
       <parameter name="PivotEnergy" scale="1e6" value="0.3" min="0.01" max="1000.0" free="0"/>
    </spectrum>
    <spatialModel type="PointSource">
      <parameter name="RA" scale="1.0" value="83.6331" min="-360" max="360" free="0"/>
      <parameter name="DEC" scale="1.0" value="22.0145" min="-90" max="90" free="0"/>
    </spatialModel>
  </source>
  <source name="CTABackgroundModel" type="CTAIrfBackground" instrument="CTA">
    <spectrum type="PowerLaw">	
      <parameter name="Prefactor" scale="1.0" value="1.0" min="1e-3" max="1e+3" free="1"/>	
      <parameter name="Index" scale="1.0" value="0.0" min="-5.0" max="+5.0" free="1"/>
      <parameter name